<h2> Image scraping from Pinterest </h2>

In [1]:
import time
import requests
import bs4
import shutil # to save it locally
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import pandas as pd

In [26]:
#function to scrape images from pinterest infinite scrolling webpages

def scrape_img_urls(pinterest_url):
    options = ChromeOptions()
    options.headless = True
    driver = Chrome(options = options)
    
    driver.get(pinterest_url)
    
    time.sleep(1)    # Allow 1 seconds for the web page to open
    scroll_pause_time = 2  # You can set your own pause time. My laptop is a bit slow so I use 2 sec
    screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
    
    i=0
    urls = []
    
    # simulating inifinite scrolling
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo({screen_height}*{i}, {screen_height}*{j});".format(screen_height=screen_height, i=i,j=i+1))  
        i += 1
        time.sleep(scroll_pause_time)
        
        soup = bs4.BeautifulSoup(driver.page_source,'lxml')
        image_tags = soup.find_all('img',class_='hCL kVc L4E MIw')
        for image in image_tags:
            urls.append(image['src'])
        
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break
    
    Image_urls = set(urls)
    
    return list(Image_urls)
    

In [27]:
#function to download the images to a local drive

def download_images(urls,file_path):
    for url in urls:
        img_file = url.split("/")[-1]
        r = requests.get(url, stream = True) # Open the url image, set stream to True, this will return the stream content.
        if r.status_code == 200:
            r.raw.decode_content = True      # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            filename = file_path + img_file
            with open(filename,'wb') as f:   # Open a local file with wb ( write binary ) permission
                    shutil.copyfileobj(r.raw, f)  

In [28]:
url_memes ='https://www.pinterest.com/mohammedshafrandj/english-memes/'
url_quotes = 'https://in.pinterest.com/sreevenireddy/english-quotes/'

In [29]:
meme_urls = scrape_img_urls(url_memes)
quote_urls = scrape_img_urls(url_quotes)

In [33]:
file_path = "C:\\Users\\subhr\\Desktop\\Quotes\\"
download_images(quote_urls,file_path)

In [34]:
file_path = "C:\\Users\\subhr\\Desktop\\Memes\\"
download_images(meme_urls,file_path)